# 6장: 예지력 (단계별 사고)

- [수업](#lesson)
- [연습 문제](#exercises)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 도우미 함수를 설정하세요.

In [ ]:
!pip install anthropic

# Python의 내장 정규 표현식 라이브러리 가져오기
import re
import anthropic

# IPython 저장소에서 API_KEY 및 MODEL_NAME 변수 검색
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## 수업

누군가가 당신을 깨워서 즉시 여러 복잡한 질문을 하고 바로 대답해야 한다면, 어떻게 하시겠습니까? 아마도 **먼저 답변을 생각할 시간**이 주어졌을 때보다 잘 하지 못할 것입니다.

짐작하셨나요? Claude도 마찬가지입니다.

**Claude에게 단계별로 생각할 시간을 주면 때때로 Claude의 정확도가 향상**됩니다. 특히 복잡한 작업에서 그렇습니다. 그러나 **생각은 소리내어 할 때만 의미가 있습니다**. Claude에게 생각하되 답변만 출력하라고 요청할 수 없습니다. 이 경우 실제로 생각이 이루어지지 않았습니다.

### 예제

아래 프롬프트에서, 두 번째 문장이 첫 번째 문장을 거짓으로 만든다는 것이 인간 독자에게는 명확합니다. 그러나 **Claude는 "관련 없는"이라는 단어를 너무 문자 그대로 받아들입니다**.

In [ ]:
# 프롬프트
PROMPT = """이 영화 리뷰의 감정이 긍정적인가요 부정적인가요?

이 영화는 신선함과 독창성으로 내 마음을 사로잡았습니다. 완전히 관련 없는 소식으로, 저는 1900년부터 바위 밑에서 살고 있습니다."""

# Claude의 응답 출력
print(get_completion(PROMPT))

Claude의 응답을 개선하기 위해, **Claude가 답변하기 전에 먼저 생각할 수 있도록 해봅시다**. Claude가 작업을 처리하고 생각하는 단계를 문자 그대로 설명함으로써 이를 수행합니다. 역할 프롬프팅을 약간 추가하면, Claude가 리뷰를 더 깊이 이해할 수 있게 됩니다.

In [ ]:
# 시스템 프롬프트
SYSTEM_PROMPT = "당신은 영화 리뷰를 잘 읽는 사람입니다."

# 프롬프트
PROMPT = """이 리뷰의 감정이 긍정적인가요 부정적인가요? 먼저, <positive-argument>와 <negative-argument> XML 태그 안에 각 측면에 대한 최선의 주장을 작성한 다음, 답변하세요.

이 영화는 신선함과 독창성으로 내 마음을 사로잡았습니다. 완전히 관련 없는 소식으로, 저는 1900년부터 바위 밑에서 살고 있습니다."""

# Claude의 응답 출력
print(get_completion(PROMPT, SYSTEM_PROMPT))

**Claude는 때때로 순서에 민감합니다**. 이 예제는 Claude가 미묘한 텍스트를 이해하는 능력의 경계에 있으며, 이전 예제에서 주장의 순서를 부정적인 것이 먼저, 긍정적인 것이 두 번째로 바꾸면 Claude의 전체 평가가 긍정적으로 바뀝니다.

대부분의 상황에서(하지만 모든 상황에서는 아니며, 혼란스럽게도), **Claude는 두 가지 옵션 중 두 번째를 선택할 가능성이 더 높습니다**. 아마도 웹에서 가져온 훈련 데이터에서 두 번째 옵션이 정답일 가능성이 더 높았기 때문일 것입니다.

In [ ]:
# 프롬프트
PROMPT = """이 리뷰의 감정이 부정적인가요 긍정적인가요? 먼저 <negative-argument>와 <positive-argument> XML 태그 안에 각 측면에 대한 최선의 주장을 작성한 다음, 답변하세요.

이 영화는 신선함과 독창성으로 내 마음을 사로잡았습니다. 관련 없는 이야기지만, 저는 1900년부터 바위 밑에서 살고 있습니다."""

# Claude의 응답 출력
print(get_completion(PROMPT))

**Claude가 생각할 수 있게 하면 Claude의 답변이 잘못된 것에서 올바른 것으로 바뀔 수 있습니다**. Claude가 실수하는 많은 경우에서 이것은 그렇게 간단합니다!

Claude의 답변이 잘못된 예를 살펴보고 Claude에게 생각하도록 요청하면 어떻게 그것을 수정할 수 있는지 살펴보겠습니다.

In [ ]:
# 프롬프트
PROMPT = "1956년에 태어난 배우가 출연한 유명한 영화를 말해보세요."

# Claude의 응답 출력
print(get_completion(PROMPT))

이번에는 Claude에게 단계별로 생각하도록 요청하여 이를 수정해 보겠습니다. 이번에는 `<brainstorm>` 태그를 사용합니다.

In [ ]:
# 프롬프트
PROMPT = "1956년에 태어난 배우가 출연한 유명한 영화를 말해보세요. 먼저 <brainstorm> 태그 안에 몇몇 배우와 그들의 출생 연도에 대해 브레인스토밍한 다음, 답변을 제공하세요."

# Claude의 응답 출력
print(get_completion(PROMPT))

---

## 연습 문제

- [연습 문제 6.1 - 이메일 분류하기](#exercise-61---classifying-emails)
- [연습 문제 6.2 - 이메일 분류 형식 지정](#exercise-62---email-classification-formatting)

### 연습 문제 6.1 - 이메일 분류하기

이 연습에서는 Claude에게 이메일을 다음 카테고리로 분류하도록 지시할 것입니다:

- (A) 판매 전 질문
- (B) 고장 또는 결함 있는 제품
- (C) 결제 질문
- (D) 기타 (설명해주세요)

연습의 첫 번째 부분에서는 `PROMPT`를 변경하여 **Claude가 올바른 분류만 출력하도록** 만드세요. 답변에는 **올바른 선택의 문자(A - D)와 괄호, 그리고 카테고리 이름이 포함**되어야 합니다.

`EMAILS` 목록의 각 이메일 옆에 있는 주석을 참조하여 해당 이메일이 어떤 카테고리로 분류되어야 하는지 확인하세요.

In [ ]:
# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = """이 이메일을 녹색 또는 파란색으로 분류해주세요: {email}"""

# Claude의 응답 미리 채우기(있는 경우)
PREFILL = ""

# 목록으로 저장된 변수 내용
EMAILS = [
    "안녕하세요 -- 제 Mixmaster4000이 작동할 때 이상한 소리가 납니다. 또한 전자제품이 타는 것 같은 연기와 플라스틱 냄새가 약간 납니다. 교체품이 필요합니다.", # (B) 고장 또는 결함 있는 제품
    "Mixmaster 4000을 음식 혼합뿐만 아니라 페인트 혼합에도 사용할 수 있나요?", # (A) 판매 전 질문 또는 (D) 기타 (설명해주세요)
    "취소한 후 4개월 동안 월별 요금이 끝나기를 기다리고 있습니다!! 무슨 일이 일어나고 있는 거죠???", # (C) 결제 질문
    "어떻게 여기 왔는지 모르겠어요. 컴퓨터를 잘 다루지 못합니다. 도와주세요." # (D) 기타 (설명해주세요)
]

# 이메일당 여러 개의 올바른 분류 가능성을 수용하기 위해 목록의 목록으로 저장된 올바른 분류
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 정규식 채점에 사용할 각 카테고리의 문자열 값 사전
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# 이메일 목록을 반복
for i,email in enumerate(EMAILS):
    
    # 이메일 텍스트를 이메일 자리 표시자 변수에 대체
    formatted_prompt = PROMPT.format(email=email)
   
    # Claude의 응답 가져오기
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Claude의 응답 채점
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Claude의 응답 출력
    print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude의 응답 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 채점 ------------------------------------------")
    print("이 연습 문제가 올바르게 해결되었습니다:", grade, "\n\n\n\n\n\n")

❓ 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_6_1_hint; print(exercise_6_1_hint)

여전히 어렵다면, 아래 셀을 실행하여 예제 솔루션을 확인하세요.

In [ ]:
from hints import exercise_6_1_solution; print(exercise_6_1_solution)

### 연습 문제 6.2 - 이메일 분류 형식 지정

이 연습에서는 위 프롬프트의 출력을 정확히 원하는 방식으로 형식을 지정하도록 개선할 것입니다.

좋아하는 출력 형식 지정 기술을 사용하여 Claude가 올바른 분류의 문자만 `<answer></answer>` 태그로 감싸도록 만드세요. 예를 들어, 첫 번째 이메일에 대한 답변에는 정확히 `<answer>B</answer>` 문자열이 포함되어야 합니다.

각 이메일에 대한 올바른 문자 카테고리가 무엇인지 잊어버린 경우 `EMAILS` 목록 옆의 주석을 참조하세요.

In [ ]:
# 변수 내용의 자리 표시자가 있는 프롬프트 템플릿
PROMPT = """이 이메일을 녹색 또는 파란색으로 분류해주세요: {email}"""

# Claude의 응답 미리 채우기(있는 경우)
PREFILL = ""

# 목록으로 저장된 변수 내용
EMAILS = [
    "안녕하세요 -- 제 Mixmaster4000이 작동할 때 이상한 소리가 납니다. 또한 전자제품이 타는 것 같은 연기와 플라스틱 냄새가 약간 납니다. 교체품이 필요합니다.", # (B) 고장 또는 결함 있는 제품
    "Mixmaster 4000을 음식 혼합뿐만 아니라 페인트 혼합에도 사용할 수 있나요?", # (A) 판매 전 질문 또는 (D) 기타 (설명해주세요)
    "취소한 후 4개월 동안 월별 요금이 끝나기를 기다리고 있습니다!! 무슨 일이 일어나고 있는 거죠???", # (C) 결제 질문
    "어떻게 여기 왔는지 모르겠어요. 컴퓨터를 잘 다루지 못합니다. 도와주세요." # (D) 기타 (설명해주세요)
]

# 이메일당 여러 개의 올바른 분류 가능성을 수용하기 위해 목록의 목록으로 저장된 올바른 분류
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 정규식 채점에 사용할 각 카테고리의 문자열 값 사전
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# 이메일 목록을 반복
for i,email in enumerate(EMAILS):
    
    # 이메일 텍스트를 이메일 자리 표시자 변수에 대체
    formatted_prompt = PROMPT.format(email=email)
   
    # Claude의 응답 가져오기
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Claude의 응답 채점
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Claude의 응답 출력
    print("--------------------------- 변수 대체가 있는 전체 프롬프트 ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude의 응답 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 채점 ------------------------------------------")
    print("이 연습 문제가 올바르게 해결되었습니다:", grade, "\n\n\n\n\n\n")

❓ 힌트가 필요하면 아래 셀을 실행하세요!

In [ ]:
from hints import exercise_6_2_hint; print(exercise_6_2_hint)

### 축하합니다!

지금까지 모든 연습 문제를 해결했다면, 다음 장으로 넘어갈 준비가 되었습니다. 즐거운 프롬프팅 되세요!